# (one-step) Biot equation

In this tutorial we present how to solve the (static) Biot equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$, the flux $q$ and the pressure $p$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, $K$ the permeability of the porous medium and $s_0$ its storativity, and $\alpha$ the Biot-Willis coefficient fot the coupling between the two models.
We want to solve the following problem: find $({u}, {q}, p)$ such that
$$
    \begin{aligned}
         & \nabla \cdot [2 \mu {\epsilon}({u}) + \lambda \nabla \cdot u I
         - \alpha p {I}] = -{b}   \\
         & \mu{q} + {K}\nabla p = {0}                        \\
         & \partial_t (s_0 p + \alpha \nabla \cdot {u}) +
        \nabla \cdot {q} = \psi
    \end{aligned}
    \quad \text{in } \Omega.
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The stress tensor, which can be post-processed from $u$, is given by
$$
    \sigma = 2 \mu \epsilon(u) + \lambda \nabla \cdot u I - \alpha p I
$$

In particular, we consider only the spatial structure of the previous problem by doing only one time step. We call the latter $\Delta t$ and write the discrete problem as
$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        \alpha D & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u \\ q \\ p
    \end{bmatrix}
    =
    \begin{bmatrix}
        b \\ 0 \\ \psi
    \end{bmatrix}
$$
where $K$ is the stiffness matrix associated with the elatic problem, $D$ is the coupling between the two physics, $M_q$ is the mass matrix associated to the flux variable, $B$ is the divergence matrix for the flow problem, and $M_p$ is the mass matrix associated with the pressure. All the aforementioned matrices are properly scaled by their physical parameters if not explicitly written. The second row has been multiply by $\Delta t$ to preserve the skew-symmetry of the problem.

## Exercise 1: fixed-strain iterative solver

To solve the discrete problem we employ the fixed-strain iterative solver which reads:
given $u^i$ compute $(u^{i+1}, q^{i+1}, p^{i+1})$ by solving until convergence the following steps:

$$
 \begin{bmatrix}
         \Delta t M_q & - \Delta t B^\top \\
         \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
         q^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    \begin{bmatrix}
       0 \\ \psi- D\alpha u^{i}
    \end{bmatrix}
$$

followed by
$$
K u^{i+1} = b + \alpha D^\top p^{i+1}
$$

Note that i) we are performing only one time step starting from $u=0$, $p=0$, therefore the right hand side is simplified and ii) the two system are equivalent to the following block triangular one:

$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        0 & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u^{i+1} \\ q^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    -
    \begin{bmatrix}
        0 & 0 & 0 \\
        0 & 0 & 0 \\
        \alpha D& 0 & 0
    \end{bmatrix}
    \begin{bmatrix}
        u^{i} \\ q^{i} \\ p^{i}
    \end{bmatrix}
    +
    \begin{bmatrix}
        b \\ 0 \\ \psi
    \end{bmatrix}
$$

A footing problem is when a force is impose on the top compressing the body and the bottom is fixed. In this case we assume that the force is applied on the central portion of the top boundary.

For this test case we set $\Omega = [0, 3] \times [0, 1]$, $b = 0$, and the following boundary conditions:
$$ 
\begin{aligned}
    &u = 0 &&\text{ and }&& \nu \cdot q = 0 &&\text{ on } \partial \Omega  \setminus \partial_{top} \Omega
\\
&\nu \cdot \sigma = 0 &&\text{ and }&& p = 0 &&\text{ on } \partial_{top} \Omega \setminus \partial_s \Omega
\\
&\nu \cdot \sigma = [0, -1]^\top &&\text{ and }&& p = 0 &&\text{ on }\partial_s \Omega 
\end{aligned}
$$
where $\partial_{s} \Omega = [1, 2] \times \{1\}$.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [17]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg


# useful for the stopping criteria in the iterative solvers
def compute_err(x, x_i, mass):
    diff = np.sqrt((x_i - x) @ mass @ (x_i - x))
    norm = np.sqrt(x_i @ mass @ x_i)
    return diff / norm if norm else diff

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ and Raviart-Thomas for the flux $q$, we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [18]:
mesh_size = 0.05
delta_t = 0.01
dim = 2

nx = np.array([30, 10])
physdims = np.ones(2)
sd = pp.grids.simplex.StructuredTriangleGrid(nx, physdims)
sd.nodes[0, :] *= 3
sd.compute_geometry()

# data for the iterative solvers
tol = 1e-4
max_iter = 1e2

With the following code we set the data and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [19]:
key = "biot"

# the physical parameters of the problem, assumed constant
lambda_ = 1
mu = 0.5
alpha = 1
s0 = 0.1 # 1 0.1

data = {}
param = {"second_order_tensor": pp.SecondOrderTensor(np.ones(sd.num_cells)),"lambda": lambda_, "mu": mu}
pp.initialize_data(sd, data, key, param)

# selection of the boundary conditions
bd_q = sd.tags["domain_boundary_faces"]
bd_q[np.isclose(sd.face_centers[1, :], 1)] = False

bd_u = sd.tags["domain_boundary_nodes"]
bd_u[np.isclose(sd.nodes[1, :], 1)] = False
bd_u = np.hstack([bd_u] * dim)

top_s = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, :], 1),
        sd.face_centers[0, :] > 1,
        sd.face_centers[0, :] < 2,
    )
)

fun = lambda _: np.array([0, -1])

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [20]:
# definition of the discretizations
vec_p1 = pg.VecLagrange1(key)
p0 = pg.PwConstants(key)
rt0 = pg.RT0(key)

# dofs for the different variables
dof_u = sd.num_nodes * dim
dof_q = sd.num_faces
dof_p = sd.num_cells
dofs = np.cumsum([dof_u, dof_q, dof_p])

# construction of the block matrices
mass_u = vec_p1.assemble_mass_matrix(sd)
mass_q = rt0.assemble_mass_matrix(sd)
mass_p = p0.assemble_mass_matrix(sd)
div_q = mass_p @ rt0.assemble_diff_matrix(sd)

sym_sym = vec_p1.assemble_symgrad_symgrad_matrix(sd)
div_div = vec_p1.assemble_div_div_matrix(sd)
div_u = mass_p @ vec_p1.assemble_div_matrix(sd)

stiff_u = 2 * mu * sym_sym + lambda_ * div_div


We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [21]:
# construction of the matrices for the sub_problems and the RHS (the fixed part)

#TODO matrix for the fluid problem
sp_fluid = sps.bmat(
    [[delta_t * mass_q, -delta_t * div_q.T], [delta_t * div_q, s0 * mass_p]]
)
#TODO matrix for the mechanical problem
sp_mech = stiff_u

bd_mech = vec_p1.assemble_nat_bc(sd, fun, top_s)
bd_fluid = np.hstack((bd_q, np.zeros(dof_p, dtype=bool)))

step = 0
err = tol + 1

# initialization of the solution
u_i = np.zeros(dof_u)
q_i = np.zeros(dof_q)
p_i = np.zeros(dof_p)

while err > tol and step < max_iter:
    # TODO for a given u solve the flow problem
    rhs_fluid = 

    ls1 = pg.LinearSystem(sp_fluid, rhs_fluid)
    ls1.flag_ess_bc(bd_fluid, np.zeros_like(rhs_fluid))
    x = ls1.solve()
    # split of the solution from the vector x
    q, p = np.split(x, [dof_q])

    # TODO for a given p solve the mech problem
    rhs_mech = 

    ls2 = pg.LinearSystem(sp_mech, rhs_mech)
    ls2.flag_ess_bc(bd_u, np.zeros_like(rhs_mech))
    u = ls2.solve()

    # compute the stopping criteria
    step += 1
    err_u = compute_err(u_i, u, mass_u)
    err_q = compute_err(q_i, q, mass_q)
    err_p = compute_err(p_i, p, mass_p)
    
    # update the solution
    u_i = u.copy()
    q_i = q.copy()
    p_i = p.copy()

    err = err_u + err_q + err_p

    print(step, err, err_u, err_q, err_p)

1 1.0 1.0 0.0 0.0
2 3.6418888767770508 1.6418888767770508 1.0 1.0
3 4.686491005788077 1.288442769268133 1.7024970803739157 1.6955511561460281
4 4.092136172819659 1.457882885303998 1.3156476553408656 1.3186056321747956
5 4.290097046359712 1.389207965888376 1.4507765937832249 1.4501124866881114
6 4.204385231755549 1.4175958377374867 1.3934401747024177 1.393349219315645
7 4.23838902428813 1.4060271860864266 1.4163098725363483 1.4160519656653556
8 4.224308387628593 1.4107887154567593 1.4068229234004748 1.406696748771359
9 4.230024617770636 1.408856898553517 1.4106366832767656 1.4105310359403533
10 4.227666489974408 1.4096602196031638 1.4090335744176967 1.4089726959535476
11 4.228621994226599 1.4093397941681962 1.409662440377176 1.409619759681227
12 4.228224629989335 1.4094766172307611 1.4093870183974817 1.4093609943610925
13 4.228383385577128 1.4094242661402572 1.409488146181185 1.4094709732556858
14 4.228315834586231 1.4094480637614235 1.4094392348235913 1.4094285360012158
15 4.2283418369

We post-process now the stress tensor, one `sd.dim` x `sd.dim` tensor per cell.

In [22]:
cell_sigma = vec_p1.compute_stress(sd, u, data)

# split the tensor in each component for the exporting
cell_sigma_xx = cell_sigma[:, 0, 0]
cell_sigma_xy = cell_sigma[:, 0, 1]
cell_sigma_yy = cell_sigma[:, 1, 1]

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. Similarly for the flow field $q$. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [23]:
# compute the cell displacement, one vector per cell
proj = vec_p1.eval_at_cell_centers(sd)
cell_u = proj @ u
# we need to add the z component for the exporting
cell_u = np.hstack((cell_u, np.zeros(sd.num_cells)))
cell_u = cell_u.reshape((3, -1))

# compute the cell flow, one vector per cell
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q * q).reshape((3, -1), order="F")
cell_p = p0.eval_at_cell_centers(sd) * p

# export the final solution
save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu(
    [
        ("cell_u", cell_u),
        ("cell_q", cell_q),
        ("cell_p", cell_p),
        ("cell_sigma_xx", cell_sigma_xx),
        ("cell_sigma_xy", cell_sigma_xy),
        ("cell_sigma_yy", cell_sigma_yy),
    ]
)

In [24]:
#compute the spectral radius of the iteration matrix
#preconditioner P
P = sps.bmat(
    [
        [stiff_u, None, -alpha * div_u.T],
        [None, delta_t * mass_q, -delta_t * div_q.T],
        [None, delta_t * div_q, s0 * mass_p],
    ]
)

# P-A
M = sps.bmat(
    [
        [0 * stiff_u, None, None],
        [None, 0 * mass_q, None],
        [-alpha * div_u, None, 0 * mass_p],
    ]
)
# compute the 6 largest magnitude eigs for the generalized problem M x = lambda P x
ll = sps.linalg.eigs(M, 6, P, None, "LM")
print(ll)

(array([-2.21994763+0.j, -1.86458025+0.j, -1.46136017+0.j, -1.11224052+0.j,
       -0.8604677 +0.j, -0.84401687+0.j]), array([[-1.06336490e+00+0.j, -7.10055377e-02+0.j,  1.01566032e-01+0.j,
         9.72125142e-02+0.j, -6.08030685e-01+0.j,  7.29830338e-02+0.j],
       [-1.06276641e+00+0.j, -6.95396996e-02+0.j,  9.90102754e-02+0.j,
         9.36806591e-02+0.j, -6.06700677e-01+0.j,  7.70757609e-02+0.j],
       [-1.05919554e+00+0.j, -6.18222687e-02+0.j,  8.74349025e-02+0.j,
         7.99849761e-02+0.j, -6.01564262e-01+0.j,  9.06123047e-02+0.j],
       ...,
       [ 2.92944639e-04+0.j, -5.56860948e-04+0.j, -7.60763901e-04+0.j,
         8.74807004e-04+0.j, -4.23017677e-04+0.j,  8.79470013e-04+0.j],
       [ 1.27147379e-04+0.j, -2.48236927e-04+0.j, -3.54478391e-04+0.j,
         4.34216852e-04+0.j, -1.81204581e-04+0.j,  4.77187359e-04+0.j],
       [ 1.28908753e-04+0.j, -2.50262739e-04+0.j, -3.53984442e-04+0.j,
         4.27477739e-04+0.j, -1.92692694e-04+0.j,  4.59738657e-04+0.j]]))
